In [1]:
# Install required packages if not already installed
!pip install ipywidgets pandas numpy scikit-learn matplotlib seaborn plotly scipy


   ---------------------------------------- 0.0/914.9 kB ? eta -:--:--
   ---------------------------------------- 914.9/914.9 kB 16.7 MB/s  0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------------------- 2.2/2.2 MB 29.8 MB/s  0:00:00
   ---------------------------------------- 0.0/9.9 MB ? eta -:--:--
   ---------------------------------------  9.7/9.9 MB 63.1 MB/s eta 0:00:01
   ---------------------------------------- 9.9/9.9 MB 41.5 MB/s  0:00:00

   ---------------------------------------- 0/4 [widgetsnbextension]
   ---------- ----------------------------- 1/4 [plotly]
   ---------- ----------------------------- 1/4 [plotly]
   ---------- ----------------------------- 1/4 [plotly]
   ---------- ----------------------------- 1/4 [plotly]
   ---------- ----------------------------- 1/4 [plotly]
   ---------- ----------------------------- 1/4 [plotly]
   ---------- ----------------------------- 1/4 [plotly]
   ---------- -----

In [2]:
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# For interactive widgets
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ All libraries imported successfully!")


✅ All libraries imported successfully!


In [3]:
def load_all_models():
    """Load all trained Random Forest models"""
    models = {}
    model_dir = Path("models")
    
    if not model_dir.exists():
        print("❌ Models directory not found. Please ensure models are trained and saved in 'models/' folder")
        return models
    
    for model_file in model_dir.glob("*.pkl"):
        category = model_file.stem.replace("_model", "")
        try:
            with open(model_file, 'rb') as f:
                models[category] = pickle.load(f)
            print(f"✅ Loaded model: {category}")
        except Exception as e:
            print(f"❌ Error loading {category}: {e}")
    
    return models

def load_feature_weights():
    """Load optimized feature weights"""
    weights_file = Path("optimized_weights.pkl")
    if weights_file.exists():
        with open(weights_file, 'rb') as f:
            weights = pickle.load(f)
        print(f"✅ Loaded feature weights")
        return weights
    else:
        print("⚠️ No optimized weights found")
        return {}

# Load models and weights
models = load_all_models()
weights = load_feature_weights()

print(f"\n📊 Total models loaded: {len(models)}")
print(f"📋 Available categories: {list(models.keys())}")


✅ Loaded model: badminton
✅ Loaded model: baseball
✅ Loaded model: collectivault_all
✅ Loaded model: cricket_bats
✅ Loaded model: cricket_medals
✅ Loaded model: cricket_other_items
✅ Loaded model: f1_racing
✅ Loaded model: hockey
⚠️ No optimized weights found

📊 Total models loaded: 8
📋 Available categories: ['badminton', 'baseball', 'collectivault_all', 'cricket_bats', 'cricket_medals', 'cricket_other_items', 'f1_racing', 'hockey']


In [6]:
def convert_value_to_rating(value):
    """Convert predicted value to 1-10 rating scale"""
    if value < 100:
        return 1
    elif value < 500:
        return 2
    elif value < 1000:
        return 3
    elif value < 2500:
        return 4
    elif value < 5000:
        return 5
    elif value < 10000:
        return 6
    elif value < 25000:
        return 7
    elif value < 50000:
        return 8
    elif value < 100000:
        return 9
    else:
        return 10

def get_rating_description(rating):
    """Get description for rating"""
    descriptions = {
        1: "⭐ Basic Item - Common collectible",
        2: "⭐⭐ Below Average - Limited interest",
        3: "⭐⭐⭐ Average - Standard collectible",
        4: "⭐⭐⭐⭐ Good - Decent item",
        5: "⭐⭐⭐⭐⭐ Above Average - Notable piece",
        6: "⭐⭐⭐⭐⭐⭐ Very Good - Quality collectible",
        7: "⭐⭐⭐⭐⭐⭐⭐ Excellent - Premium item",
        8: "⭐⭐⭐⭐⭐⭐⭐⭐ Outstanding - Rare collectible",
        9: "⭐⭐⭐⭐⭐⭐⭐⭐⭐ Exceptional - Museum quality",
        10: "⭐⭐⭐⭐⭐⭐⭐⭐⭐⭐ Legendary - Investment grade"
    }
    return descriptions.get(rating, "Unknown")

def predict_rating(category, features_dict):
    """Make prediction using loaded models"""
    if category not in models:
        return None, None, "Model not available for this category"
    
    try:
        model = models[category]
        
        # Create feature DataFrame
        feature_df = pd.DataFrame([features_dict])
        
        # Apply log transformation for positive numeric features
        numeric_cols = feature_df.select_dtypes(include=[np.number]).columns
        for col in numeric_cols:
            if (feature_df[col] > 0).all():
                feature_df[f'{col}_log'] = np.log1p(feature_df[col])
        
        # Make prediction
        predicted_value = model.predict(feature_df)[0]
        rating = convert_value_to_rating(predicted_value)
        
        return rating, predicted_value, "Success"
    
    except Exception as e:
        return None, None, f"Error: {str(e)}"

print("✅ Helper functions defined successfully!")
def predict_rating(category, features_dict):
    """Make prediction using loaded models"""
    if category not in models:
        return None, None, "Model not available for this category"
    
    try:
        model = models[category]
        feature_df = pd.DataFrame([features_dict])
        
        # NO log transformation - model was trained without it
        predicted_value = model.predict(feature_df)[0]
        rating = convert_value_to_rating(predicted_value)
        
        return rating, predicted_value, "Success"
    except Exception as e:
        return None, None, f"Error: {str(e)}"



✅ Helper functions defined successfully!


In [8]:
# Output widget
output = widgets.Output()

# Category dropdown
category_dropdown = widgets.Dropdown(
    options=list(models.keys()) if models else ['No models loaded'],
    description='Category:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

# Feature sliders - MATCHED TO YOUR DATA
condition_slider = widgets.IntSlider(
    value=70, min=0, max=100, step=5,
    description='Condition Score:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

signed_slider = widgets.IntSlider(
    value=70, min=0, max=100, step=10,
    description='Signed Score:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

worn_slider = widgets.IntSlider(
    value=70, min=0, max=100, step=5,
    description='Worn Score:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

historical_slider = widgets.IntSlider(
    value=70, min=0, max=100, step=5,
    description='Historical Score:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

market_slider = widgets.IntSlider(
    value=70, min=0, max=100, step=5,
    description='Market Demand:',
    style={'description_width': '150px'},
    layout=widgets.Layout(width='500px')
)

# Buttons
predict_button = widgets.Button(
    description='🔍 Predict Rating',
    button_style='success',
    layout=widgets.Layout(width='200px', height='40px')
)

reset_button = widgets.Button(
    description='🔄 Reset',
    button_style='warning',
    layout=widgets.Layout(width='200px', height='40px')
)

def on_predict(b):
    with output:
        clear_output()
        
        # Collect features - EXACT MATCH TO TRAINED MODEL
        features = {
            'ConditionScore': condition_slider.value,
            'SignedScore': signed_slider.value,
            'WornScore': worn_slider.value,
            'HistoricalScore': historical_slider.value,
            'MarketDemandScore': market_slider.value
        }
        
        # Predict
        rating, value, msg = predict_rating(category_dropdown.value, features)
        
        if rating:
            display(HTML(f"""
            <div style='background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); 
                        padding: 30px; border-radius: 15px; color: white;'>
                <h2 style='text-align: center; margin: 0;'>🏆 Prediction Results</h2>
                <hr style='border-color: white;'>
                <h1 style='text-align: center; font-size: 80px; margin: 20px 0;'>{rating}/10</h1>
                <p style='text-align: center; font-size: 24px;'>{get_rating_description(rating)}</p>
                <hr style='border-color: white;'>
                <h3 style='text-align: center;'>Estimated Value: £{value:,.2f}</h3>
                <p style='text-align: center;'>Category: {category_dropdown.value}</p>
            </div>
            """))
            
            # Visualization
            fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
            
            # Rating bar
            ax1.barh(['Rating'], [rating], color='#667eea', height=0.5)
            ax1.set_xlim(0, 10)
            ax1.set_title('Rating Score', fontsize=16, fontweight='bold')
            ax1.grid(alpha=0.3)
            
            # Features
            names = list(features.keys())
            values = list(features.values())
            colors = plt.cm.viridis(np.linspace(0, 1, len(names)))
            ax2.barh(names, values, color=colors)
            ax2.set_title('Input Features', fontsize=16, fontweight='bold')
            ax2.grid(alpha=0.3)
            
            plt.tight_layout()
            plt.show()
        else:
            display(HTML(f"<p style='color: red; font-size: 18px;'>❌ {msg}</p>"))

def on_reset(b):
    condition_slider.value = 70
    signed_slider.value = 70
    worn_slider.value = 70
    historical_slider.value = 70
    market_slider.value = 70
    
    with output:
        clear_output()
        print("✅ Reset complete!")

predict_button.on_click(on_predict)
reset_button.on_click(on_reset)

# Display GUI
display(HTML("""
<h1 style='color: #667eea; text-align: center; font-family: Arial, sans-serif;'>
    🏏 Collectivault Rating System
</h1>
<p style='text-align: center; font-size: 18px; color: #666;'>
    AI-Powered Sports Memorabilia Valuation
</p>
<hr style='border: 3px solid #667eea;'>
"""))

display(widgets.VBox([
    widgets.HTML("<h3 style='color: #FF9800;'>📋 Select Category</h3>"),
    category_dropdown,
    widgets.HTML("<h3 style='color: #FF9800;'>⚙️ Enter Item Scores</h3>"),
    condition_slider,
    signed_slider,
    worn_slider,
    historical_slider,
    market_slider,
    widgets.HTML("<br>"),
    widgets.HBox([predict_button, reset_button]),
    widgets.HTML("<br>"),
    output
]))

print("✅ Interactive GUI loaded successfully!")


✅ Interactive GUI loaded successfully!
